# Excution time measurements


In [124]:
cycles = 10

# Pandas

In [18]:
import pandas as pd
import sys
import os
import timeit

In [118]:
data1 = 'output/data1.csv'
data2 = 'output/data2.csv'



file_pathss = [
    'output/data1.csv',
    'output/data2.csv',
    'output/data3.csv',
    'output/data4.csv'
]


file_paths = [
    'output/data1.csv',
    'output/data2.csv',
    'output/data3.csv',
    'output/data4.csv',
    'output/data5.csv',
    'output/data6.csv',
    'output/data7.csv',
    'output/data8.csv',
    'output/data9.csv',
    'output/data10.csv',
    'output/data12.csv',
    'output/data13.csv',
    'output/data14.csv',
    'output/data15.csv',
]

bigger_files = [
    'output/data16.csv',
    'output/data17.csv',
    'output/data18.csv',
    'output/data19.csv',
    'output/data20.csv',
    'output/data21.csv',
    'output/data22.csv',
    'output/data23.csv',
    'output/data24.csv'
]


def average_time(myList):
    total_time_spark = 0
    for i in myList:
        total_time_spark += i
    avg_time = total_time_spark / cycles
    return avg_time

In [30]:
df1 = pd.read_csv(data1)


# Data Loading

In [127]:
def read_csv(x):
    df = pd.read_csv(x)
    return df

def read_time(dis):
    start_time = timeit.default_timer()
    dataframe = read_csv(dis)
    end_time = timeit.default_timer()
    return end_time - start_time


def measure_average_read_time(files, cycles=10):
    avg_times = []
    for file_path in files:
        timings_pandas = [read_time(file_path) for _ in range(cycles)]
        avg_time_pandas = average_time(timings_pandas)
        avg_times.append(avg_time_pandas)
    return avg_times

def run_loading(paths):
    avg_times = measure_average_read_time(paths)

    for i, avg_time in enumerate(avg_times, 1):
        print(f"Average read time for data_{i}: {avg_time:.5f} seconds")
        
run_loading(file_paths)


In [119]:
# Measuring lead time of bigger datasize - 6Gb - 15 GB
run_loading(bigger_files)

Average read time for data_1: 25.89928 seconds
Average read time for data_2: 39.33730 seconds
Average read time for data_3: 48.52571 seconds
Average read time for data_4: 60.61468 seconds
Average read time for data_5: 70.24155 seconds
Average read time for data_6: 76.79467 seconds
Average read time for data_7: 83.14161 seconds
Average read time for data_8: 94.47625 seconds
Average read time for data_9: 106.77703 seconds


# Filtering and Selection

In [123]:
test = pd.read_csv("output/data1.csv")

addresses = ["output/data1.csv"]

# we are filtering from a column "5.164162635803223" which are in values b/n and including 6 and 7.
def filter_and_select(filter_it):
    return filter_it.loc[(filter_it["5.164162635803223"] >= 6) & (filter_it["5.164162635803223"] <= 7)]

# Measurement - Filter_csv
def filter_time(dis):
    start_time = timeit.default_timer()
    dataframe = filter_and_select(dis)
    end_time = timeit.default_timer()
    return end_time - start_time

def measure_average_filter_time(files, cycles=10):
    avg_times = []
    for file_path in files:
        local_df = pd.read_csv(file_path)
        timings_pandas = [filter_time(local_df) for _ in range(cycles)]
        avg_time_pandas = average_time(timings_pandas)
        avg_times.append(avg_time_pandas)
    return avg_times

def run_filter(addresses):
    avg_times = measure_average_filter_time(addresses)

    for i, avg_time in enumerate(avg_times, 1):
        if i >= 11:
            print(f"Average filter time for data-{i + 1}: {avg_time:.5f} seconds")
        print(f"Average filter time for data-{i}: {avg_time:.5f} seconds")
        
#run_filter(file_paths)
print(cycles)

20


run_filter(bigger_files)

# Data Transformation

In [110]:
test = pd.read_csv("output/data1.csv")

addresses = ["output/data1.csv"]


# we are filtering from a column "5.164162635803223" which are in values b/n and including 6 and 7.
def transform(filter_it):
    return filter_it.round(3)

# Measurement - Filter_csv
def transform_time(dis):
    start_time = timeit.default_timer()
    dataframe = transform(dis)
    end_time = timeit.default_timer()
    return end_time - start_time

def measure_average_transform_time(files, cycles=10):
    avg_times = []
    for file_path in files:
        local_df = pd.read_csv(file_path)
        timings_pandas = [transform_time(local_df) for _ in range(cycles)]
        avg_time_pandas = average_time(timings_pandas)
        avg_times.append(avg_time_pandas)
    return avg_times

def run_transformation(addresses):
    avg_times = measure_average_transform_time(addresses)

    for i, avg_time in enumerate(avg_times, 1):
        if i >= 11:
            print(f"Average Transform time for data-{i + 1}: {avg_time:.5f} seconds")
        print(f"Average Transform time for data-{i}: {avg_time:.5f} seconds")
        
run_transformation(file_paths)

Average Transform time for data-1: 0.02265 seconds
Average Transform time for data-2: 0.03163 seconds
Average Transform time for data-3: 0.03230 seconds
Average Transform time for data-4: 0.06181 seconds


In [120]:
hi


NameError: name 'hi' is not defined

In [7]:
import pyspark
import timeit
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, mean, count, udf
import timeit
from pyspark.sql.types import StringType
from pyspark.sql import functions as F